In [1]:
# Set up and dependencies
from pprint import pprint
import pandas as pd
import numpy as np
import re


In [2]:
# Load source file and preview
headlines = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)
headlines.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
# Examine headlines for potential issues
for ind, row in headlines.head(25).iterrows():
    print(row['headline'])

former versace store clerk sues over secret 'black code' for minority shoppers
the 'roseanne' revival catches up to our thorny political mood, for better and worse
mom starting to fear son's web series closest thing she will have to grandchild
boehner just wants wife to listen, not come up with alternative debt-reduction ideas
j.k. rowling wishes snape happy birthday in the most magical way
advancing the world's women
the fascinating case for eating lab-grown meat
this ceo will send your kids to school, if you work for his company
top snake handler leaves sinking huckabee campaign
friday's morning email: inside trump's presser for the ages
airline passengers tackle man who rushes cockpit in bomb threat
facebook reportedly working on healthcare features and apps
north korea praises trump and urges us voters to reject 'dull hillary'
actually, cnn's jeffrey lord has been 'indefensible' for a while
barcelona holds huge protest in support of refugees
nuclear bomb detonates during rehearsal 

In [4]:
def punctuationToText(row):
    text = str(row['headline'])
    text = text.replace(',',   ' pcomma')
    text = text.replace('.',   ' pperiod ')
    text = text.replace('!',   ' pexclamation')
    text = text.replace('?',   ' pquestion')
    text = text.replace(" '",  ' pquote ')
    text = text.replace("'s ", ' ppossession ')
    text = text.replace("'",   '')
    text = text.replace("(",   ' pparen ')
    text = text.replace(")",   '')
    text = text.replace(':',   ' pcolon')
    text = text.replace(';',   ' psemic')
    text = text.replace('-',   ' pdash ')
    text = text.replace('  ',  ' ')
    return text


In [5]:
headlines['head_char'] = headlines.apply(punctuationToText, axis=1)
for ind, row in headlines.head(25).iterrows():
    print(row['head_char'])

former versace store clerk sues over secret pquote black code for minority shoppers
the pquote roseanne revival catches up to our thorny political mood pcomma for better and worse
mom starting to fear son ppossession web series closest thing she will have to grandchild
boehner just wants wife to listen pcomma not come up with alternative debt pdash reduction ideas
j pperiod k pperiod rowling wishes snape happy birthday in the most magical way
advancing the world ppossession women
the fascinating case for eating lab pdash grown meat
this ceo will send your kids to school pcomma if you work for his company
top snake handler leaves sinking huckabee campaign
friday ppossession morning email pcolon inside trump ppossession presser for the ages
airline passengers tackle man who rushes cockpit in bomb threat
facebook reportedly working on healthcare features and apps
north korea praises trump and urges us voters to reject pquote dull hillary
actually pcomma cnn ppossession jeffrey lord has be

In [6]:
headlines.to_csv('text_headlines.csv', index=False)
headlines.head()

,article_link,headline,is_sarcastic,head_char
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0,former versace store clerk sues over secret pq...
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0,the pquote roseanne revival catches up to our ...
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1,mom starting to fear son ppossession web serie...
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1,boehner just wants wife to listen pcomma not c...
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0,j pperiod k pperiod rowling wishes snape happy...


In [7]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_check = stopwords.words('english')

#print(stop_check)
def stopclear(row):
    line = str(row['head_char']).split()
    newline = ' '.join([word for word in line if word not in stop_check])
    #print(newline)
    return newline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chien\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
headlines['head_stop'] = headlines.apply(stopclear, axis=1)
stop_df = headlines[['is_sarcastic', 'head_stop']]
stop_df.head()

,is_sarcastic,head_stop
0,0,former versace store clerk sues secret pquote ...
1,0,pquote roseanne revival catches thorny politic...
2,1,mom starting fear son ppossession web series c...
3,1,boehner wants wife listen pcomma come alternat...
4,0,j pperiod k pperiod rowling wishes snape happy...


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split


In [10]:
train, test = train_test_split(stop_df, test_size=0.2, random_state=7)

In [11]:
classifier = Pipeline([('ct_vect', CountVectorizer()),
                       ('tf_idf', TfidfTransformer()),
                       ('sgd', SGDClassifier())])

In [12]:
classifier.fit(train['head_stop'], train['is_sarcastic'])

C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Pipeline(memory=None,
     steps=[('ct_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

In [13]:
import sklearn.metrics as skm
test_results = classifier.predict(test['head_stop'])
test_score = skm.f1_score(test['is_sarcastic'], test_results)

In [14]:
print(f'F1 score of test set was: {test_score}')

F1 score of test set was: 0.7769389155799588


In [15]:
def inputToText(text):
    text = text.replace(',',   ' pcomma')
    text = text.replace('.',   ' pperiod ')
    text = text.replace('!',   ' pexclamation')
    text = text.replace('?',   ' pquestion')
    text = text.replace(" '",  ' pquote ')
    text = text.replace("'s ", ' ppossession ')
    text = text.replace("'",   '')
    text = text.replace("(",   ' pparen ')
    text = text.replace(")",   '')
    text = text.replace(':',   ' pcolon')
    text = text.replace(';',   ' psemic')
    text = text.replace('-',   ' pdash ')
    text = text.replace('  ',  ' ')
    return text.lower()

# user_in = input("What are you thinking?  ")

# user_out = classifier.predict([inputToText(user_in)])
# if user_out[0] == 1:
#     print("Are you being sarcastic?")
# else:
#     print("Seriously?")


In [16]:
user_in = input("What are you thinking?  ")
user_format = inputToText(user_in)
print(user_format)
user_out = classifier.predict([user_format])
if user_out[0] == 1:
    print("Bazinga!")
else:
    print("...ok...")
# print(user_out)


What are you thinking?  this project was so very extraordinarily exciting
this project was so very extraordinarily exciting
...ok...


In [17]:
# done = 0
# test_lines = []
# while (done < 10):
#     test_lines.append(inputToText(input("Line to test: ")))
#     done = done + 1

Line to test: this is not funny
Line to test: its really really not funny
Line to test: why are you laughing
Line to test: are you not entertained
Line to test: its the end of the world as we know it
Line to test: i feel fine
Line to test: wish you would step back from that ledge my friend
Line to test: you could cut ties with all the lies that youve been living in
Line to test: if you do not want to see me again, I would understand
Line to test: the angry boy a bit too insane icing over a secret pain


In [18]:
# done = 0
# test_lines = []
# while (done < 10):
#     test_lines.append(inputToText(input("Line to test: ")))
#     done = done + 1
    

Line to test: i make her smile like a drug for you
Line to test: do whatever what you want to do coming over you
Line to test: keep on smiling what we go through
Line to test: wont stop to the rhythm that divides you
Line to test: i speak to you like a chorus to a verse
Line to test: chop out another line like a coda with a curse
Line to test: something like a freak show takes the stage
Line to test: we give them the game we play , she says
Line to test: i want something else to get me through this semi charmed kind of life
Line to test: i want something else , im not listening when you say good bye


In [19]:
# test_outs = classifier.predict(test_lines)
# print(test_outs)

[0 0 1 0 1 1 0 0 0 0]


In [20]:
headlines.drop(['article_link'], axis=1, inplace=True)
headlines.head()

,headline,is_sarcastic,head_char,head_stop
0,former versace store clerk sues over secret 'b...,0,former versace store clerk sues over secret pq...,former versace store clerk sues secret pquote ...
1,the 'roseanne' revival catches up to our thorn...,0,the pquote roseanne revival catches up to our ...,pquote roseanne revival catches thorny politic...
2,mom starting to fear son's web series closest ...,1,mom starting to fear son ppossession web serie...,mom starting fear son ppossession web series c...
3,"boehner just wants wife to listen, not come up...",1,boehner just wants wife to listen pcomma not c...,boehner wants wife listen pcomma come alternat...
4,j.k. rowling wishes snape happy birthday in th...,0,j pperiod k pperiod rowling wishes snape happy...,j pperiod k pperiod rowling wishes snape happy...


In [22]:
cols = ['title', 'publication', 'headline', 'bias', 'lean', 'is_sarcastic', 'head_char']
huffpo = headlines.loc[headlines['is_sarcastic'] == 0]
huffpo['publication'] = 'Huffington Post'
huffpo['bias'] = 'LL'
huffpo['lean'] = 'L'
huffpo['title'] = huffpo['headline']
df_huffpo = huffpo[cols]
df_huffpo.to_csv('Huffington Post_headlines.csv')
print(len(df_huffpo))
df_huffpo.head()

C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is 

14985


,title,publication,headline,bias,lean,is_sarcastic,head_char
0,former versace store clerk sues over secret 'b...,Huffington Post,former versace store clerk sues over secret 'b...,LL,L,0,former versace store clerk sues over secret pq...
1,the 'roseanne' revival catches up to our thorn...,Huffington Post,the 'roseanne' revival catches up to our thorn...,LL,L,0,the pquote roseanne revival catches up to our ...
4,j.k. rowling wishes snape happy birthday in th...,Huffington Post,j.k. rowling wishes snape happy birthday in th...,LL,L,0,j pperiod k pperiod rowling wishes snape happy...
5,advancing the world's women,Huffington Post,advancing the world's women,LL,L,0,advancing the world ppossession women
6,the fascinating case for eating lab-grown meat,Huffington Post,the fascinating case for eating lab-grown meat,LL,L,0,the fascinating case for eating lab pdash grow...


In [24]:
tonion = headlines.loc[headlines['is_sarcastic'] == 1]
tonion['publication'] = 'The Onion'
tonion['bias'] = 'NA'
tonion['lean'] = 'N'
tonion['title'] = tonion['headline']
df_tonion = tonion[cols]
df_tonion.to_csv('The Onion_headlines.csv')
print(len(df_tonion))
df_tonion.head()

C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\chien\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

11724


,title,publication,headline,bias,lean,is_sarcastic,head_char
2,mom starting to fear son's web series closest ...,The Onion,mom starting to fear son's web series closest ...,NA,N,1,mom starting to fear son ppossession web serie...
3,"boehner just wants wife to listen, not come up...",The Onion,"boehner just wants wife to listen, not come up...",NA,N,1,boehner just wants wife to listen pcomma not c...
8,top snake handler leaves sinking huckabee camp...,The Onion,top snake handler leaves sinking huckabee camp...,NA,N,1,top snake handler leaves sinking huckabee camp...
15,nuclear bomb detonates during rehearsal for 's...,The Onion,nuclear bomb detonates during rehearsal for 's...,NA,N,1,nuclear bomb detonates during rehearsal for pq...
16,cosby lawyer asks why accusers didn't come for...,The Onion,cosby lawyer asks why accusers didn't come for...,NA,N,1,cosby lawyer asks why accusers didnt come forw...


In [35]:
# def specialCharCheck(row):
#     chars = str(row['head_char'])
    
#     chars = chars.replace("[^A-Za-z0-9 ]", "")
#     if chars != '':
#         print(chars)

In [36]:
# headlines['exceptions'] = headlines.apply(specialCharCheck, axis=1)

former versace store clerk sues over secret pquote black code for minority shoppers
the pquote roseanne revival catches up to our thorny political mood pcomma for better and worse
mom starting to fear son ppossession web series closest thing she will have to grandchild
boehner just wants wife to listen pcomma not come up with alternative debt pdash reduction ideas
j pperiodk pperiod rowling wishes snape happy birthday in the most magical way
advancing the world ppossession women
the fascinating case for eating lab pdash grown meat
this ceo will send your kids to school pcomma if you work for his company
top snake handler leaves sinking huckabee campaign
friday ppossession morning email pcolon inside trump ppossession presser for the ages
airline passengers tackle man who rushes cockpit in bomb threat
facebook reportedly working on healthcare features and apps
north korea praises trump and urges us voters to reject pquote dull hillary
actually pcomma cnn ppossession jeffrey lord has bee

butt crack bandit caught on camera holding duo at gunpoint
exxon mobil told to hand over decades of climate documents in major legal blow
here ppossession the pquote hocus pocus remake you never knew you wanted
justin trudeau is king of the political sock game no more
planet explodes
news roundup for august 29 pcomma 2017
fighting and airstrikes continue throughout yemen as dialogue remains distant
these chicago landmarks are open to the public for the first time in decades
justin bieber invites controversy with cornrow pic
males circumcised to reduce hiv risk in mozambique shift gender norms surrounding sex
massachusetts is offering a model for how doctors can talk to their patients about guns
cruz likely to block trump on a second ballot at gop convention
middle east crisis traced to trouble pdash making genie
the pquote artisanal school supplies list
robert mueller ascends into sky with umbrella after trump family promises they learned lesson about honesty
5 things to do the minute 

4 identity protection habits every college student should have
price of gas rises to four expletives per gallon
here ppossession what the oscar nominations should look like
this unreleased britney spears song is all kinds of sultry
insufferable prick distinctly said no cilantro
heidi cruz gets a boost in new york from trump pquote nastiness backlash
why top talent is passing your company by
department of education study finds only 30% of students adequately prepared for spring musical
u pperiods pperiod general jealous that syrian army allowed to attack citizens
younger siblings are good for older siblings health
giuliana rancic on cheating ex jerry oconnell pcolon pquote it ppossession all good
lena dunham has a theory on why men pcomma apparently pcomma dont like serena williams and ronda rousey
conceptual genius goes as self for halloween
nation demands more slow pdash motion footage of syrup cascading onto pancakes
high pdash culture wars heat up over controversial new opera
paul s

god sick of new angel ppossession annoying fucking voice
british crew member dies during round the world yacht race
trump and xi in mar pdash a pdash lago pcolon a tweetless summit would be a win
taylor swift thanks her pquote boyfriend adam during iheartradio music awards speech
tearful gun manufacturers thankful they all made it out of massacre safely
assistant manager accused of sexual indiscrimination
area man dead of fries
toy story 4 coming to theaters in 2019
leonard cohen at 80
fareed zakaria and u2 for president
jimmy kimmel hilariously stops by pquote sesame street to introduce a new letter
um pcomma reese witherspoon ppossession look pdash alike daughter is stunning
facebook user verifies truth of article by carefully checking it against own preconceived opinions
sources pcolon petraeus knew about affair for more than a year
world ppossession worst person decides to go into marketing
biden winks after offering to buy eggnog for white house christmas party
this rescued mink j

x pdash files creator hints reboot may confirm that theory about scully
fourth pdash grader with shark tooth necklace must have killed great white
there ppossession finally a museum devoted to telling the story of hbcus
senate passes 3 pdash year highway funding bill
your neighbors pcolon should you consider talking to them pquestion
first 10 minutes of chess game spent explaining replacement pieces
progressives in congress call for $2 trillion in infrastructure spending
heartless dutch curators put deranged scrawlings of mentally ill suicide victim on full display for world to mock
nra pcolon pquote please try to remember all the wonderful things guns do for us every day
governor too embarrassed to say which state he leads
report pcolon women only made up 2 pperiod7% of video game bosses last year
despite madaya aid pcomma u pperiodn pperiod still fails to end country ppossession sieges
blog post read by mother to shape child ppossession next 18 years
9 shows that make the case for wa

...

tearful elon musk warns about dangers of ai after having heart broken by beautiful robotrix
how to register voters in a south carolina jail
handwriting expert confirms killer used cursive
3 pcomma500 pdash year pdash old dagger was used as a doorstop
study pcolon 73% of bedroom closets have wife ppossession boy toy crouched naked inside
myth of white supremacy is now out in the open
the fda should even the score for women ppossession sexual health
what to expect when youre expecting a couch
study finds eating doctor after birth can provide essential nutrients to new mothers
watch this comedy editor hilariously mockument his return to standup
missouri gov pperiod eric greitens refuses to resign despite calls from his own party
how much will black lives matter in trump ppossession america pquestion
queer icon kate bornstein reflects on queer and trans identity in 2015
bernie kicks into overdrive in nh
bill clinton pcolon trump attacks on hillary clinton pquote fact pdash free
im no longe

new department of interior program to reduce deer population by providing free condoms to fawns
these illustrations perfectly sum up what it ppossession like to have anxiety
the end of shared sacrifice set in stone pcolon yale as metaphor
gunmen in eqypt mosque attack carried isis flag pcomma prosecutor says
winn pdash dixie pcolon from one to a thousand pdash a journey of a hundred years
empire star jussie smollett reminds us that aids isnt a problem of the past
bill maher gets real about marijuana legalization
10 communication secrets of great leaders
new countries pcomma who ppossession first pquestion
our smog standards are in jeopardy under trump pcomma and we need to fight back
national forest service recommends campers tie up their food to avoid attracting other visitors
fed up zara workers battle for more hours
north carolina residents terrified after hearing state passed new law
senator from troubled home state repeatedly acting out in congress
10 signs it ppossession time for

negative parent pdash teacher conference not exactly eye pdash opening for area mother
teen choice awards honor cory monteith with posthumous surfboard
check it out pcolon deer
cancerous tumor befriends small boy
havana ppossession forgotten baseball team played a key role in u pperiods pperiod pdash cuba relations
but she can talk pquestion
barry jenkins quietly makes history with oscar nomination trifecta
nationwide art project is making space for historic women in all 50 states
liberal male hypocrisy pcomma modern day rasputins and the culture of deceit
transgender community caught slightly off guard by baskin pdash robbins enthusiastic support
cvs is reportedly in talks to acquire aetna
terrible artist thinks latest piece really represents a culmination of everything he ppossession been working toward all his life
trump praises chuck schumer in reposted tweet that first called him pquote cunning
supreme court rules in favor of most buck pdash wild pride parade nation ppossession ev

powerball officials remove plastic balls from pig urine brine
chris martin says he and gwyneth paltrow are pquote very close
jilted bride penned the world ppossession most depressing ebay listing
man taking photo with ipad oblivious to how badass he looks
shell pdash shocked cnbc staffers had long flight home
just pdash opened factory to create 250 new jobs pcomma 170 new cancer cases
tv critics admit to never having watched the wire
after 4 years living in asia pcomma im suddenly a minority again  pparenand it sucks
pakistani husbands can pquote lightly beat their wives pcomma islamic council says
ghost hunters enjoys surprising 100% success rate
friend attempting to provide comfort has no clue what the fuck she ppossession talking about
right side of fish tank where all the action at
the pquote westworld mystery that started it all might finally be solved
32 gifts people with anxiety really want for the holidays
the billionaire journalist
gop health care bill disinvests in women ppos

lady gaga introduces us to her creepy pquote american horror story pcolon hotel children
kung fu master with iron crotch is one ballsy martial artist
philip morris ceo forces senator to dance for his amusement
one woman pcomma two proposals pcomma dumbfounding indecision
a guide to sex at 50 and beyond
dining out in dallas by john mariani
experts advise against throwing laptop across office even though it will feel incredible
affable detective with healthy personal life hasnt solved case in months
nation not sure how many ex pdash trump staffers it can safely reabsorb
cdc rolls out fleet of narcan biplanes to fumigate opioid pdash ravaged small towns
stephen colbert gets women from 1776 to react to the first female presidential nominee
this is how you wear shoes with ankle straps
michael bolton singing john bolton ppossession scary words about war is almost soothing
how the grammys gloss over great indigenous music being made today
two conferences spotlight the muslim world ppossession